In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from netneurotools import cluster
from netneurotools import modularity

import networkx as nx
from sknetwork.clustering import Louvain, get_modularity
from sknetwork.data import from_edge_list

In [9]:
# DNp01 (giant fiber) to DNp11
body_ids = ["2307027729","5813024015", "1565846637", "1405231475", "1466998977", "5813023322", "1100404581", "1226887763", "1228264951", "512851433", "5813026936", "1281324958"]
DNp_ids = [int(i) for i in body_ids]

In [3]:
consensusResults = np.load("consensusResults.npy", allow_pickle=True)

In [4]:
# adj_mat = np.loadtxt("adj_mat.csv", delimiter=",")
all_connection_df = pd.read_csv("all_connection_df.csv")
dfFilt = all_connection_df[['bodyId_pre', 'bodyId_post', 'weight']] # sknetwork uses 3rd col as weight

In [5]:
# only needed bc Louvain's clustering assignment output is based on graph.names order
graph = from_edge_list(list(dfFilt.itertuples(index=False)), weighted=True, directed=True) # without directed=True, wrong # of elements

In [6]:
graph.adjacency, graph.names

(<5596x5596 sparse matrix of type '<class 'numpy.int64'>'
 	with 389065 stored elements in Compressed Sparse Row format>,
 array([ 326253554,  357245785,  357249472, ..., 7112615304, 7112622763,
        7112624834]))

In [7]:
# create directed, weighted networkx graph from dataframe of edges & weights
# NOTE: it seems that NetworkX conversion shuffles edge ordering (so clustering results may be different)
G = nx.from_pandas_edgelist(all_connection_df, 'bodyId_pre', 'bodyId_post', 'weight', create_using=nx.DiGraph())

In [8]:
G.number_of_nodes(), G.number_of_edges()

(5596, 389065)

In [78]:
# for each consensus result, for each cluster, extract its subgraph
# calculate graph theory metrics (average betweenness centrality, modularity, small world) for each subgraph

for result in consensusResults:
    consensus = result[1]
    for clusterInd in range(1, consensus.max()+1): # cluster indices are beween 1-5 (inclusive)
        cluster_indices = np.where(consensus == clusterInd)[0]
        cluster_nodes = [graph.names[i] for i in cluster_indices]
        cluster_subgraph = G.subgraph(cluster_nodes)
        # print(cluster_subgraph.number_of_nodes(), cluster_subgraph.number_of_edges())

        # calculate graph theory metrics
        # betweenness centrality
        if clusterInd == 4:
            # cluster_betweenness = nx.betweenness_centrality(cluster_subgraph, weight='weight') # maybe set endpoints=True
            # cluster_degreeCentrality = nx.degree_centrality(cluster_subgraph)
            # cluster_eigenCentrality = nx.eigenvector_centrality(cluster_subgraph, weight='weight')
            # degAssortCoeff = nx.degree_assortativity_coefficient(cluster_subgraph, weight='weight')
            # avgDegCon = nx.average_degree_connectivity(cluster_subgraph, weight='weight')
            # groupBetCentrality = nx.group_betweenness_centrality(G, cluster_nodes, weight='weight')
            # groupDegreeCentrality = nx.group_degree_centrality(G, cluster_nodes)

            # smallWorld = nx.algorithms.smallworld.sigma(cluster_subgraph.to_undirected())
            
            # cluster_modularity = nx.community.modularity(G, cluster_nodes, weight='weight')
            break
        # modularity
    break
    print()

KeyboardInterrupt: 

In [74]:
groupDegreeCentrality

0.482007996446024

In [65]:
sum(avgDegCon.values())/len(avgDegCon.values()) # doesn't seem meaningful

145.67808078312498

In [64]:
degAssortCoeff

-0.0840938857573405

In [51]:
sum(cluster_degreeCentrality.values()) / len(cluster_degreeCentrality)

0.07173788325575241

In [50]:
sum(cluster_eigenCentrality.values()) / len(cluster_eigenCentrality)

0.008290702240462593

In [53]:
sum(cluster_betweenness.values()) / len(cluster_betweenness)

0.004125238268360178

In [57]:
cluster_betweenness[2307027729], cluster_betweenness[5813024015] # DNp01,02

(0.036588253808819825, 0.045981048093354494)

In [41]:
### find subgraph consisting of nodes in cluster 1
# first extract the nodes associated with cluster 1
consensus250 = consensusResults[0][1] # consensus clustering labels of each of the 5596 nodes for iteration 250
cluster1_nodeIndices = np.where(consensus250 == 4)[0] # indices of cluster1 neurons based on graph.names
cluster1_nodes = [graph.names[i] for i in cluster1_nodeIndices] # get cluster1 neuron bodyIds from graph.names

# then extract the subgraph
cluster1_subgraph = G.subgraph(cluster1_nodes)

In [42]:
cluster1_subgraph.number_of_nodes(), cluster1_subgraph.number_of_edges()

(1094, 42890)

In [26]:
# find betweenness centrality of each node in subgraph